<a href="https://colab.research.google.com/github/ClementPla/DeepFiberQ/blob/main/DeepFiberQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions
Create a data folder on the left panel and upload your images in it.

The accepted formats are tif or czi. But czi seems buggy, so tif preferred for now.

## GPUs

If available, we recommend using a GPU (available on the top right, in the dropdown, select "Change runtime type)"

In [1]:
!pip uninstall -y dnafiber
!pip install -q git+https://github.com/ClementPla/DeepFiberQ

Found existing installation: dnafiber 0.1.950
Uninstalling dnafiber-0.1.950:
  Successfully uninstalled dnafiber-0.1.950
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from dnafiber.data.utils import read_czi, read_tiff
# If you are using jpeg or png files, we expect them to already be processed. 
# You can simply read them with PIL or OpenCV.
from dnafiber.deployment import _get_model, MODELS_ZOO, format_results
from dnafiber.inference import infer
from dnafiber.postprocess.core import refine_segmentation


In [ ]:
ROOT_IMG = "PATH_TO_YOUR_IMAGE_FILE.CZI"  # or .TIFF # or .JPEG or .PNG
pixel_size = 0.13 # Pixel size in micrometers
device = "cuda"  # or "cpu"
# You may want to iterate through your images and process them in a loop.
# For example, if you have multiple images in a directory, you can use glob or os.listdir to get the file paths.

img = read_czi(ROOT_IMG)  # or read_tiff(ROOT_IMG)
model = _get_model(MODELS_ZOO["SegFormer MiT-B4"]+ "_finetuned") # Load the pre-trained model. 
# This default model is trained on the SegFormer MiT-B4 architecture, but you can choose any other model from the zoo. 
# However, it should be the best choice for most images. The postfix "_finetuned" indicates that the model has been fine-tuned for DNA fiber segmentation. 

predicted = infer(model, img, scale=pixel_size, device=device) 
# This may take a while depending on the size of the image and your GPU.
refined = refine_segmentation(predicted)
dataframe = format_results(refined, pixel_size=pixel_size)

dataframe.to_csv("results.csv", index=False)  # Save results to a CSV file

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
